# PyCaret Pipeline for PD-L1 Prediction

This notebook demonstrates the PyCaret-based machine learning pipeline for predicting PD-L1 expression from radiomics features.

## Workflow:
1. Load and prepare data
2. Feature selection using stability selection
3. Model training and comparison
4. Model evaluation on hold-out test set

In [ ]:
import pycaret
from pycaret.classification import *
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from tableone import TableOne
import pandas as pd
from collections import defaultdict 

## 1. Data Loading

⚠️ **USER ACTION REQUIRED**: Load your radiomics features and labels here.

Expected data format:
- `feature_train`: DataFrame with radiomics features (rows=patients, columns=features)
- `feature_test`: DataFrame with test set features
- Target column: `encoded_PDL1` with values {0, 1, 2} for PD-L1 categories

Example:
```python
feature_train = pd.read_csv('./data/train_features.csv', index_col=0)
feature_test = pd.read_csv('./data/test_features.csv', index_col=0)
```

In [ ]:
# Load your training data here
# feature_train = pd.read_csv('./data/train_features.csv', index_col=0)
# feature_test = pd.read_csv('./data/test_features.csv', index_col=0)

# Prepare data
data = feature_train
data['encoded_PDL1'] = data['encoded_PDL1'].astype(int)

## 2. Feature Selection with Stability Selection

Use stability selection with Logistic Regression to identify robust features.

In [ ]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from joblib import Parallel, delayed

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score
import plotly.graph_objects as go

# --------- Functions ---------

def compute_validation_scores_and_coefficients(
    X_train, y_train, X_val, y_val,
    n_cs=15, random_state=3457
):
    """
    Train on X_train, validate on X_val to select optimal C.
    Also computes coefficient paths for visualization.
    Parallelizes the C-loop across cores.
    """
    C_values     = np.logspace(-3, 3, n_cs)
    log_C_values = np.log10(C_values)
    feature_names = X_train.columns.tolist()

    def eval_one_C(idx, C):
        pipe = make_pipeline(
            StandardScaler(),
            LogisticRegression(
                C=C, penalty='l1', solver='saga',
                multi_class='multinomial', max_iter=10_000,
                random_state=random_state, n_jobs=1
            )
        )
        pipe.fit(X_train, y_train)
        val   = log_loss(y_val, pipe.predict_proba(X_val))
        coef  = pipe.named_steps['logisticregression'].coef_.copy()
        return val, coef

    results = Parallel(n_jobs=-1, verbose=5)(
        delayed(eval_one_C)(i, C) for i, C in enumerate(C_values)
    )

    val_scores, coef_paths = zip(*results)
    val_scores = np.array(val_scores)               # (n_cs,)
    coef_paths = np.stack(coef_paths, axis=0)       # (n_cs, n_classes, n_features)

    best_idx     = val_scores.argmin()
    optimal_C    = C_values[best_idx]
    optimal_loss = val_scores[best_idx]

    return {
        'optimal_C':      optimal_C,
        'optimal_loss':   optimal_loss,
        'C_values':       C_values,
        'log_C_values':   log_C_values,
        'val_scores':     val_scores,
        'coef_paths':     coef_paths,
        'feature_names':  feature_names
    }

def fit_final_model(X, y, C, random_state=3457):
    """
    Fit on X,y with given C, return pipeline, selected features, and coef matrix.
    """
    pipe = make_pipeline(
        StandardScaler(),
        LogisticRegression(
            C=C, penalty='l1', solver='saga',
            multi_class='multinomial', max_iter=10_000,
            random_state=random_state, n_jobs=-1
        )
    )
    pipe.fit(X, y)
    coefs = pipe.named_steps['logisticregression'].coef_          # (n_classes, n_features)
    mask  = np.any(coefs != 0, axis=0)
    selected = X.columns[mask].tolist()
    return pipe, selected, coefs


X_full = pd.concat([X_train, X_val], axis=0)
y_full = pd.concat([y_train, y_val], axis=0)

outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
n_inner  = 2
n_cs     = 20

# Directory for saving plots
NESTED_OUTPUT_DIR = ""
os.makedirs(NESTED_OUTPUT_DIR, exist_ok=True)

# --------- Outer folds in parallel ---------
outer_splits = list(enumerate(outer_cv.split(X_full, y_full), 1))

def process_outer(fold_tuple):
    fold_idx, (tr_idx, te_idx) = fold_tuple
    X_tr, X_te = X_full.iloc[tr_idx], X_full.iloc[te_idx]
    y_tr, y_te = y_full.iloc[tr_idx], y_full.iloc[te_idx]

    # inner split for hyperparameter tuning
    inner = StratifiedKFold(n_splits=n_inner, shuffle=True, random_state=fold_idx+123)
    itr, ivl = next(inner.split(X_tr, y_tr))
    X_ti, X_vi = X_tr.iloc[itr], X_tr.iloc[ivl]
    y_ti, y_vi = y_tr.iloc[itr], y_tr.iloc[ivl]

    # tune C
    res = compute_validation_scores_and_coefficients(X_ti, y_ti, X_vi, y_vi, n_cs=n_cs)

    # fit on outer-train with its best C, extract features
    pipe, sel, _ = fit_final_model(X_tr, y_tr, res['optimal_C'])

    return {
        'val_scores':    res['val_scores'],
        'coef_paths':    res['coef_paths'],
        'feature_names': res['feature_names'],
        'C_values':      res['C_values'],       # <— now included
        'log_C_values':  res['log_C_values'],   # <— now included
        'optimal_C':     res['optimal_C'],
        'selected':      sel
    }

# run all outer folds in parallel
fold_results = Parallel(n_jobs=-1, verbose=5)(
    delayed(process_outer)(fs) for fs in outer_splits
)

# collect
all_vs             = [fr['val_scores']    for fr in fold_results]
all_coefs          = [fr['coef_paths']    for fr in fold_results]
selected_per_fold  = [fr['selected']      for fr in fold_results]
C_grid             = fold_results[0]['C_values']
log_C_grid         = fold_results[0]['log_C_values']
feature_names      = fold_results[0]['feature_names']

# --------- Consensus Hyperparameter ---------
vs_arr      = np.stack(all_vs)            # (n_folds, n_cs)
mean_vs     = vs_arr.mean(axis=0)
std_vs      = vs_arr.std(axis=0)
best_idx    = mean_vs.argmin()
consensus_C = C_grid[best_idx]
print(f"\n→ Consensus C (mean inner‑CV): {consensus_C:.4g}")

# --------- Merged Validation Curve ---------
fig_val = go.Figure()
fig_val.add_trace(go.Scatter(
    x=np.concatenate([log_C_grid, log_C_grid[::-1]]),
    y=np.concatenate([mean_vs - std_vs, (mean_vs + std_vs)[::-1]]),
    fill='toself', fillcolor='rgba(128,128,128,0.2)',
    line=dict(color='rgba(0,0,0,0)'), showlegend=False
))
fig_val.add_trace(go.Scatter(
    x=log_C_grid, y=mean_vs, mode='lines', name='Mean Log‑Loss'
))
fig_val.update_layout(
    title="Nested CV Mean Validation Curve",
    xaxis_title="log10(C)", yaxis_title="Log‑Loss"
)
fig_val.write_html(os.path.join(NESTED_OUTPUT_DIR, "merged_validation_curve.html"))
fig_val.write_image(os.path.join(NESTED_OUTPUT_DIR, "merged_validation_curve.svg"))
fig_val.show()

# --------- Merged Coefficient Paths ---------
cp_arr  = np.stack(all_coefs)            # shape (n_folds, n_cs, n_classes, n_features)
mean_cp = cp_arr.mean(axis=0)            # (n_cs, n_classes, n_features)
fnames  = feature_names
n_classes = mean_cp.shape[1]

# pick top‑5 features by max absolute mean coeff across all classes
maxf   = np.max(np.abs(mean_cp), axis=(0,1))
topk   = np.argsort(maxf)[-3:]

fig_all = go.Figure()
for i in topk:
    for cls in range(n_classes):
        fig_all.add_trace(go.Scatter(
            x=log_C_grid,
            y=mean_cp[:, cls, i],
            mode='lines',
            name=f"{fnames[i]} (class {cls})"
        ))

fig_all.update_layout(
    title="Nested CV Mean Coefficient Paths (all classes)",
    xaxis_title="log10(C)",
    yaxis_title="Coefficient"
)
fig_all.write_html(os.path.join(NESTED_OUTPUT_DIR, "merged_coefficient_paths_all_classes.html"))
fig_all.write_image(os.path.join(NESTED_OUTPUT_DIR, "merged_coefficient_paths_all_classes.svg"))
fig_all.show()

# --------- Final Fit on All Train+Val ---------
final_pipe, final_features, _ = fit_final_model(X_full, y_full, consensus_C)
print(f"\nFinal selected features ({len(final_features)}):")
print(final_features)

# --------- Stable Features Across Folds ---------
counts      = Counter(f for fold_sel in selected_per_fold for f in fold_sel)
stable_feats = [f for f, c in counts.items() if c >= 2]  # appear in ≥2 of 3 folds
print("\nFeatures selected in ≥2 folds:")
print(stable_feats)

In [ ]:
# refit on stable_feats alone
X_stable = X_full[stable_feats]
stable_pipe, _, _ = fit_final_model(X_stable, y_full, consensus_C)

# extract and tabulate
lr2 = stable_pipe.named_steps['logisticregression']
coefs2   = lr2.coef_                     # (n_classes, len(stable_feats))
classes2 = lr2.classes_

stable_coef_df = pd.DataFrame(
    data=coefs2.T,
    index=stable_feats,
    columns=[f"class_{c}" for c in classes2]
)

stable_coef_df

In [ ]:
stable_feats.remove('pre_logarithm_firstorder_Mean')
stable_feats

In [ ]:
selected_features = stable_feats
selected_features

## 3. Clinical Data Integration

⚠️ **USER ACTION REQUIRED**: Load clinical/immunological data here.

Expected columns: Clinical factors relevant to your study

Example:
```python
clinical_data = pd.read_excel('./data/clinical_factors.xlsx', index_col=0)
```

## 4. PyCaret Setup and Model Training

In [ ]:
print('Data for Modeling: ' + str(train_data.shape))
print('Unseen Data For Predictions: ' + str(test_data.shape))

#### Holdout Test

In [ ]:
pdl1_clf = setup(data=train_data,
                 test_data=test_data,
            target='PD-L1',
            categorical_features = [''],
            ordinal_features= {'TD': [1.0, 2.0, 34]},
            remove_outliers = True,
            normalize=True, 
            normalize_method='zscore', 
            data_split_shuffle=True, 
            data_split_stratify='PD-L1',
            fold = 5,
            fold_strategy = 'stratifiedkfold',
            fold_shuffle = True, 
            session_id=3457, 
            use_gpu=True,
            max_encoding_ohe=0
            )

In [ ]:
print("Unique labels:", train_data['PD-L1'].unique())
print("Label types:", train_data['PD-L1'].dtype)

In [ ]:
print(f'Ordinal features: {pdl1_clf._fxs["Ordinal"]}')
print(f'Categorical features: {pdl1_clf._fxs["Categorical"]}')
print(f'Numeric features: {pdl1_clf._fxs["Numeric"]}')

In [ ]:
train = get_config(variable="train_transformed")
train['encoded_PDL1'].value_counts()

In [ ]:
test = get_config(variable="test_transformed")
test['encoded_PDL1'].value_counts()

In [ ]:
best = compare_models(sort='auc', n_select=3)

#### AUC for validation step

In [ ]:
lr = create_model('lr')
cv_results = pull()  # This gets the last displayed results
print(cv_results)
# fold_results = get_config('fold_results')

## 5. Model Evaluation

Evaluate the final model on the hold-out test set.

In [ ]:
from pycaret.classification import finalize_model, predict_model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# 1) Finalize & predict
final_model = finalize_model(best[0])
preds_df    = predict_model(final_model, data=test_data)

# PyCaret’s default DataFrame has:
# - 'Label'           : the predicted class
# - 'encoded_PDL1'    : the true class (if you passed it in test_data)
# - Score_0,Score_1…  : class probabilities

# 2) Extract true vs. pred
y_true = preds_df['PD-L1']
y_pred = preds_df['prediction_label']

# 3) Compute metrics
acc = accuracy_score(y_true, y_pred)
precision, recall, f1, support = precision_recall_fscore_support(
    y_true,
    y_pred,
    labels=final_model.classes_,   # ensure class order
    average=None                   # gives you per-class arrays
)

print(f"Accuracy: {acc:.4f}\n")
print("Per‑class Precision/Recall/F1:")
for cls, p, r, f, s in zip(final_model.classes_, precision, recall, f1, support):
    print(f"  Class {cls}:  P={p:.3f}, R={r:.3f}, F1={f:.3f}  (n={s})")

# 4) (Optional) One‑line summary:
print("\nOverall classification report:\n")
print(classification_report(y_true, y_pred, target_names=[
    f"PD‑L1 <1%", "PD‑L1 1–10%", "PD‑L1 >10%"  # or however you’ve named them
], digits=4))